### Multi Agent-Systems
- A team is a group of agents that work together to achieve a common goal

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)

In [3]:
model_client

#### Single Agent Approach
- A single agent is going to create a short story for us

In [4]:
from autogen_agentchat.agents import AssistantAgent

story_agent = AssistantAgent(
    name="story_agent",
    model_client=model_client,
    system_message='You are a creating writer. Generate a short story about a brave knight and a dragon.'
)

In [5]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 50 words',
        source='user'
    )
    
    result = await story_agent.run(task=task)
    
    print(result.messages[-1].content)

await test_simple_agent()

Sir Gareth, bravest knight in the realm, faced Ignis, a dragon whose scales shimmered like a thousand sunsets.  Gareth, though fearing the beast's fiery breath, charged, his sword a silver streak against the crimson sky.  He struck, not at the heart, but at the dragon's pride – its hoard. Ignis, enraged, retreated, guarding his gold.  Gareth had won, not by strength alone, but by cunning.



#### Multi Agent Team Approach
- We will be using 3 agents to now write the story, each with its own work and mastery

In [6]:
plot_agent = AssistantAgent(
    name="plot_writer",
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

In [7]:
character_agent = AssistantAgent(
    name="character_writer",
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

In [8]:
ending_agent = AssistantAgent(
    name="ending_writer",
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)

#### **RoundRobinGroupChat** 
- Is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [9]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent],
    max_turns=6
)

team

In [10]:
team.component_type

'team'

In [11]:
async def test_team():
    task = TextMessage(
        content='Write a short story a brave boy and his Pokemon. Keep it up to 50 words',
        source='user'
    )
    
    result = await team.run(task=task)
    
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')
    

await test_team()

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words
plot_writer : Ten-year-old Kai and his fiery Charmander, Ember, journeyed through Whispering Woods.  A shadowy figure stole Ember's fire stone.  Kai, fueled by Ember's weakened flames and unwavering loyalty, vowed to retrieve it, facing mischievous Impish and cunning Mightyena along the way.  Their adventure began.

character_writer : ## The Pokémon: Ember and the Shadow Syndicate

**Ember:** A Charmander with an unusually bright and fiery spirit. Unlike many of his kind, Ember possesses an innate empathy, sensing the emotions of others – both Pokémon and humans – which makes him incredibly loyal and protective. He's smaller than average for his age, but his determination makes up for it. He was found as an egg abandoned near a volcano, and Kai, having felt a strong connection to the warmth emanating from the egg, nurtured him until he hatched.  Ember’s unique ability lies not just in his flames, but in his 

#### Comparision Table
- Single-Agent vs. Multi-Agent Comparison

    | Aspect             | Single Agent                | Multi-Agent Team              |
    |--------------------|-----------------------------|-------------------------------|
    | **Creativity**     | Limited to one viewpoint    | Diverse ideas from each agent |
    | **Depth**          | Basic story elements        | Detailed plot, characters, ending |
    | **Flexibility**    | Stuck to one style          | Adaptable with specialized roles |
    | **Setup Effort**   | Simple, one agent           | More agents, team setup       |

In [12]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.

await team.reset()  # Reset the team for a new task.

async for message in team.run_stream(task="Write a short poem about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='e44302de-8646-4675-90c9-c48608c78f42' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 4, 22, 11, 53, 404428, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='07e9ebb9-77c4-4eed-ac67-81a379f34933' source='plot_writer' models_usage=RequestUsage(prompt_tokens=23, completion_tokens=78) metadata={} created_at=datetime.datetime(2025, 8, 4, 22, 11, 59, 804690, tzinfo=datetime.timezone.utc) content="The leaves, once green, now turn to red,\nA fiery blush, the branches spread.\nA crispness hangs upon the air,\nAs nature's beauty starts to share\nIts final hues, before the snow,\nA gentle hush, a peaceful flow.\nThe pumpkins gleam, a harvest bright,\nAutumn's embrace, a warm, soft light.\n" type='TextMessage'
id='da707107-21f5-413a-8dab-ed850d59d898' source='character_writer' models_usage=RequestUsage(prompt_tokens=107, completion_tokens=693) metadata={} created_at=datetime.datetime(2025, 8, 4, 22, 12, 

In [13]:
from autogen_agentchat.teams import RoundRobinGroupChat

team2 = RoundRobinGroupChat(
    participants= [plot_agent],
    max_turns=2
)

In [14]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.

await team2.reset()  # Reset the team for a new task.

async for message in team2.run_stream(task="Write a short poem about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='245502a3-ff24-456c-88a5-6bca35f3bb45' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 4, 22, 17, 2, 989365, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='87910bce-6163-4b72-9918-2c63713e1d4d' source='plot_writer' models_usage=RequestUsage(prompt_tokens=23, completion_tokens=73) metadata={} created_at=datetime.datetime(2025, 8, 4, 22, 17, 4, 211230, tzinfo=datetime.timezone.utc) content="The leaves ablaze, a fiery show,\nCrimson, gold, where green winds blow.\nCrisp air descends, a gentle chill,\nAcross the fields, a silent thrill.\nPumpkin spice, a warming scent,\nAutumn's charm, divinely sent.\nDays grow short, the nights grow long,\nNature sleeps, a peaceful song.\n" type='TextMessage'
id='fcb1ffa1-6341-4584-bb71-893f5a768632' source='plot_writer' models_usage=RequestUsage(prompt_tokens=96, completion_tokens=0) metadata={} created_at=datetime.datetime(2025, 8, 4, 22, 17, 4, 591596, tzinfo

In [15]:
message

TaskResult(messages=[TextMessage(id='245502a3-ff24-456c-88a5-6bca35f3bb45', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 4, 22, 17, 2, 989365, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='87910bce-6163-4b72-9918-2c63713e1d4d', source='plot_writer', models_usage=RequestUsage(prompt_tokens=23, completion_tokens=73), metadata={}, created_at=datetime.datetime(2025, 8, 4, 22, 17, 4, 211230, tzinfo=datetime.timezone.utc), content="The leaves ablaze, a fiery show,\nCrimson, gold, where green winds blow.\nCrisp air descends, a gentle chill,\nAcross the fields, a silent thrill.\nPumpkin spice, a warming scent,\nAutumn's charm, divinely sent.\nDays grow short, the nights grow long,\nNature sleeps, a peaceful song.\n", type='TextMessage'), TextMessage(id='fcb1ffa1-6341-4584-bb71-893f5a768632', source='plot_writer', models_usage=RequestUsage(prompt_tokens=96, completion_tokens=0), me

In [16]:
from autogen_agentchat.teams import RoundRobinGroupChat

team3 = RoundRobinGroupChat(
    participants= [plot_agent]
)

In [ ]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.

await team3.reset()  # Reset the team for a new task.

async for message in team3.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)